In [1]:
import os
# from aeon.datasets.tsc_data_lists import multivariate_equal_length
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn
from tensorflow import keras
import tensorflow as tf
import numpy as np
import time 
# from utils import geng
import logconfig
import geng
import torch
import logging
# # 修改一个已存在的环境变量
# os.environ['PATH'] = '/usr/local/cuda-11.6/bin:' + os.environ['PATH']
# os.environ['LIBRARY_PATH'] = '/usr/local/cuda-11.6/lib64:' + os.environ['LIBRARY_PATH']
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.6/lib64:'+ os.environ['LD_LIBRARY_PATH']
# 使用设置好的环境变量
os.environ['CUDA_VISIBLE_DEVICES'] = "7"
logconfig.setup_logging(dir="./")
logging.info(f"PATH:{os.environ['PATH']}\nCUDA_HOME:{os.environ['CUDA_HOME']}")
# dataset_name = "170Kailuan-relu-5"

2024-01-28 08:13:00.027127: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 08:13:00.947058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-28 08:13:02 INFO [root] [<module>:23] - PATH:/data4/conda_envs/g2/bin:/usr/local/cuda-11.6/bin:/home/beihang/.vscode-server/bin/8b3775030ed1a69b13e4f4c628c612102e30a681/bin/remote-cli:/data4/conda_envs/g2/bin:/usr/local/cuda-10.1/bin:/sbin:/usr/bin:/home/beihang/anaconda3/bin:/home/beihang/anaconda3/condabin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/usr/local/cuda/bin:/snap/bin
CUDA_HOME:/usr/local/cud

# Data process

## Example of Data

In [2]:
# from aeon.datasets import load_classification
# import numpy as np
# dataset_name = "Libras"
# archive_name="MTS"
# X, Y,meta_data = load_classification(dataset_name,return_metadata=True)
# X_reshaped = np.transpose(X, (0, 2, 1))
# X = X_reshaped
# nb_classes = len(np.unique(Y, axis=0))
# nb_classes,X.shape,Y.shape,np.unique(Y)

## Data Kailuan

In [3]:

archive_name="MTS"
time_series1 = pd.read_csv("./combined_sheets.csv")
time_series2 = pd.read_csv("./combined_sheets2.csv")
meta_data = pd.read_csv("./combinde170_info.csv")
time_series1.shape,time_series2.shape,meta_data.shape 

((140088, 6), (39711, 8), (170, 35))

In [4]:
time_series2 = time_series2.drop(['SystolicPressure','DiastolicPressure'],axis=1)
time_series = pd.concat([time_series1, time_series2], axis=0)
time_series

,Timestamp,Sleepstate,Breath,Heartrate,Movement,Source
0,2021-07-02 16:35:25,4,16,78,2,1
1,2021-07-02 16:36:25,4,16,76,0,1
2,2021-07-02 16:37:25,4,16,74,0,1
3,2021-07-02 16:38:25,4,16,74,0,1
4,2021-07-02 16:39:25,4,16,73,0,1
...,...,...,...,...,...,...
39706,2023-12-22 22:58:00,4,17,86,0,170
39707,2023-12-22 22:59:00,2,17,87,0,170
39708,2023-12-22 23:00:00,4,17,87,0,170
39709,2023-12-22 23:01:00,4,18,88,0,170


In [5]:
# Movement全变成1 二值化
# time_series['Movement'] = np.where(time_series['Movement'] != 0, 1, 0)

## 构造每个source的二维数组

In [6]:
grouped = time_series.groupby('Source')

# 提取特征列
features = [ 'Breath', 'Heartrate']

# # 将每个组转换为二维数组并堆叠
# three_dim_array = np.array([group[features].values for _, group in grouped])

# three_dim_array.shape # 显示三维数组的形状

## Movement 处理
- Movement 取消： 只有fold3有提升 其他4个fold下降
- Movement 全部+1 不行 都有下降


In [7]:
# time_groupby = time_series.groupby('Source')

In [8]:
# 查看movement的分布
# movement_stac=[{}]
# for i in range(1,136):
#     logging.info(i)
#     keys, vals = np.unique(time_groupby.get_group(int(i))["Movement"].values,return_counts=True)
#     result_dict = dict(zip(keys, vals))
#     logging.info(result_dict)
#     movement_stac.append(result_dict)

# # movement_stac

## 填充 截断 时间序列

## 填充/截断 构造数据集(去掉过少的)

In [9]:
# trial block 


In [10]:
# 重新计算每个样例的时间步数（考虑到了数据过滤）
filtered_time_steps_per_source = time_series.groupby('Source').size()

# 重新筛选出有效的 Source 过滤掉过少的
valid_sources = filtered_time_steps_per_source[filtered_time_steps_per_source > 500].index
obsolete_sources = filtered_time_steps_per_source[filtered_time_steps_per_source <= 500].index
logging.info("valid sources:", valid_sources, "\nobsolete sources:", obsolete_sources, "\n len of obsoletes:",filtered_time_steps_per_source[obsolete_sources])
# 重新获取有效的数据
valid_filtered_time_series = time_series[time_series['Source'].isin(valid_sources)]

# 重新分组
valid_filtered_grouped = valid_filtered_time_series.groupby('Source')

# 重新计算平均时间步数（此时应该没有 NaN 值）
average_time_steps = int(valid_filtered_grouped.size().mean())

logging.info(f"\n average_time_steps:{average_time_steps}")

# 初始化新的三维数组（考虑到了有效的样例数）
valid_sources_count = len(valid_sources)
three_dim_array_valid = np.zeros((valid_sources_count, average_time_steps, len(features)))

# 填充新的三维数组
for i, (source, group) in enumerate(valid_filtered_grouped):
    data = group[features].values
    current_steps = data.shape[0]
    if current_steps < average_time_steps:
        fill_values = {
            'Breath': group['Breath'].mean(),
            'Heartrate': group['Heartrate'].mean(),
        }
        fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
        full_data = np.vstack(( fill_array,data))
    else:
        # 调整为截取较后面的元素
        full_data = data[current_steps-average_time_steps:, :]
    # logging.info(f"source:{source},current_steps:{current_steps}, full_data shape:{full_data.shape}")
    three_dim_array_valid[i, :, :] = full_data


final_data = three_dim_array_valid

# 进行数据标准化
final_data = geng.standard_scaler_total(final_data)

final_data.shape

--- Logging error ---
Traceback (most recent call last):
  File "/data4/conda_envs/g2/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/data4/conda_envs/g2/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/data4/conda_envs/g2/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/data4/conda_envs/g2/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/data4/conda_envs/g2/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/data4/conda_envs/g2/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/data4/conda_envs/g2/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/data4/conda_envs/g

(164, 1088, 2)

### 标签构造
- 意识状态
- GCS

In [11]:
valid_sources

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170],
      dtype='int64', name='Source', length=164)

In [12]:
labels = meta_data["意识状态"].values

valid_sources = valid_sources-1
labels = [labels[i] for i in range(len(labels)) if i in valid_sources]
# labels = [1 if x == 2 else x for x in labels]
nb_classes = len(np.unique(labels, axis=0))
labels = np.array(labels)
labels, labels.shape

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
        2, 2, 2, 1, 1, 1, 1, 1, 2, 1]),
 (164,))

## (填充/截断)构造数据集 原本版 

In [13]:

# average_time_steps = int(grouped.apply(lambda x: x.shape[0]).mean())

# # 初始化三维数组
# features = ['Sleepstate', 'Breath', 'Heartrate', 'Movement']
# three_dim_array = np.zeros((135, average_time_steps, len(features)))

# # 填充三维数组
# for source, group in grouped:
#     data = group[features].values
#     current_steps = data.shape[0]

#     if current_steps < average_time_steps:
#         fill_values = {
#             'Breath': group['Breath'].mean(),
#             'Heartrate': group['Heartrate'].mean(),
#             'Sleepstate': group['Sleepstate'].median(),
#             'Movement': 0
#         }
#         fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
#         full_data = np.vstack((data, fill_array))
#     else:
#         full_data = data[:average_time_steps, :]

#     three_dim_array[source-1, :, :] = full_data

# three_dim_array.shape  # 显示三维数组的形状


### 标签构造
- 意识状态
- GCS

In [14]:
# labels = meta_data["意识状态"].values
# nb_classes = len(np.unique(labels, axis=0))
# labels,nb_classes

In [15]:
def data_final_process(x,y):
    nb_classes = len(np.unique(y, axis=0))
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(y.reshape(-1, 1))
    y = enc.transform(y.reshape(-1, 1)).toarray()

    # # save orignal y because later we will use binary
    # y_true = np.argmax(y_test, axis=1)
    logging.info(f'x.shape: {x.shape}, y.shape: {y.shape}\n')
    if len(x.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x = x.reshape((x.shape[0], x.shape[1], 1))

    input_shape = x.shape[1:]
    logging.info(f'input_shape: {input_shape}\n')
    
    return x,y,input_shape

In [16]:
# classifier_name="fcn"
# root_dir="/data4/gsprivate/dl-4-tsc"
# output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
#                     dataset_name + '/'
                    
# if create_directory(output_directory) is None:
#     logging.info("Creating directory:{} None".format(output_directory))

# x,y,input_shape=data_final_process(final_data,labels)
# fcn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
# logging.info(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

In [17]:
# classifier_name="cnn"
# root_dir="/data4/gsprivate/dl-4-tsc"
# output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
#                     dataset_name + '/'
                    
# if create_directory(output_directory) is None:
#     logging.info("Creating directory:{} None".format(output_directory))

# x,y,input_shape=data_final_process(final_data,labels)
# cnn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
# logging.info(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

In [18]:
import utils
class Args:
    def __init__(self):
        self.dataset = 'Kailuan'
        self.batch = 16
        self.lr = 0.01
        self.nlayers = 2
        self.emb_size = 64
        self.nhead = 8
        self.task_rate = 0.5
        self.masking_ratio = 0.15
        self.lamb = 0.8
        self.epochs = 300
        self.ratio_highest_attention = 0.5
        self.avg = 'macro'
        self.dropout = 0.01
        self.nhid = 128
        self.nhid_task = 128
        self.nhid_tar = 128
        self.task_type = 'classification'  # Or 'regression', depending on your task

# 使用这个 Args 类来创建一个 args 对象
args = Args()

# 现在你可以像在命令行环境中一样使用这些参数了
# 例如，打印学习率
logging.info(args.lr)


2024-01-28 08:13:03 INFO [root] [<module>:27] - 0.01


In [19]:
prop = utils.get_prop(args)
logging.info("prop: " , prop)

2024-01-28 08:13:03 INFO [root] [<module>:2] - prop: 


In [22]:
classifier_name="TARnet"
root_dir="/data4/gsprivate/TARNet/"
output_directory = root_dir + '/results/' + classifier_name + '/'
        
                    
if geng.create_directory(output_directory) is None:
    logging.info("Creating directory:{} None".format(output_directory))

# x,y,input_shape=data_final_process(final_data,labels)
# mcnn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
k_splits = geng.splits(final_data,labels,4)
idx = 1
for x_train_fold, y_train_fold, x_val_fold, y_val_fold in k_splits:
    logging.info(f"\n---fold:{idx}---")
    logging.info(f"x_train_fold.shape:{x_train_fold.shape}, y_train_fold.shape:{y_train_fold.shape}, x_val_fold.shape:{x_val_fold.shape}, y_val_fold.shape:{y_val_fold.shape}")
    
    X_train_task, y_train_task, X_test, y_test = utils.preprocess(prop, x_train_fold, y_train_fold, x_val_fold, y_val_fold)
    logging.info(f"X_train_task.shape:{X_train_task.shape}, y_train_task.shape:{y_train_task.shape}, X_test.shape:{ X_test.shape}, y_test.shape:{y_test.shape}")
    
    prop['nclasses'] = torch.max(y_train_task).item() + 1 if prop['task_type'] == 'classification' else None
    prop['dataset'], prop['seq_len'], prop['input_size'] = prop['dataset'], X_train_task.shape[1], X_train_task.shape[2]
    prop['device'] = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
    logging.info(f"prop:{prop}")

    idx+=1
    logging.info('Initializing model...')
    model, optimizer, criterion_tar, criterion_task, best_model, best_optimizer = utils.initialize_training(prop)
    logging.info('Model intialized...')

    logging.info('Training start...')
    utils.training(model, optimizer, criterion_tar, criterion_task, best_model, best_optimizer, X_train_task, y_train_task, X_test, y_test, prop)
    logging.info('Training complete...')

# logging.info(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

2024-01-28 08:15:53 INFO [root] [<module>:7] - Creating directory:/data4/gsprivate/TARNet//results/TARnet/ None
2024-01-28 08:15:53 INFO [root] [<module>:14] - 
---fold:1---
2024-01-28 08:15:53 INFO [root] [<module>:15] - x_train_fold.shape:(123, 1088, 2), y_train_fold.shape:(123,), x_val_fold.shape:(41, 1088, 2), y_val_fold.shape:(41,)
2024-01-28 08:15:53 INFO [root] [<module>:17] - X_train_task.shape:torch.Size([128, 1088, 2]), y_train_task.shape:torch.Size([123]), X_test.shape:torch.Size([48, 1088, 2]), y_test.shape:torch.Size([41])
2024-01-28 08:15:53 INFO [root] [<module>:21] - prop:{'batch': 16, 'lr': 0.01, 'nlayers': 2, 'emb_size': 64, 'nhead': 8, 'task_rate': 0.5, 'masking_ratio': 0.15, 'task_type': 'classification', 'lamb': 0.8, 'epochs': 300, 'ratio_highest_attention': 0.5, 'avg': 'macro', 'dropout': 0.01, 'nhid': 128, 'nhid_task': 128, 'nhid_tar': 128, 'dataset': 'Kailuan', 'nclasses': 3, 'seq_len': 1088, 'input_size': 2, 'device': device(type='cuda', index=0)}
2024-01-28 08

/data4/gsprivate/TARNet//results/TARnet/ exist


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.91 GiB total capacity; 7.00 KiB already allocated; 4.06 MiB free; 2.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# %%capture cap --no-stderr
# logging.info("这将写入到文件中")

# geng.fit_splits_for_mcnn(mcnn_classifier,x,y,epochs=250)
logging.info("DONE")

In [ ]:
# geng.fit_splits(cnn_classifier,x,y,batch_size=8,epochs=200)
# logging.info('DONE')

In [ ]:
# import torch 
import tensorflow as tf

# logging.info(torch.version.cuda)·
# logging.info(torch.backends.cudnn.version())
logging.info("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
# import os

# # Retrieving all environment variables
# env_vars = os.environ

# # Displaying the environment variables
# env_vars_dict = {key: env_vars[key] for key in env_vars}
# env_vars_dict['LD_LIBRARY_PATH']